In [49]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import torch
import json
import pandas as pd

from src.utils import get_default_pydantic_model

In [50]:
results = torch.load(paths.RESULTS_PATH/"medication/medication_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_examples_10.pt")

In [51]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(results)

In [52]:
df_broken = df[~df["successful"]]

In [53]:
# Try to fix broken JSON by searching for last "}" then adding "]"
def fix_json(json_str):
    last_index = json_str.rfind("}")
    if last_index != -1:  # If "}," is found
        fixed_json = json_str[:last_index + 1] + "]}"

        return fixed_json
    else:
        # If "}," is not found, return the original string
        return json_str

In [54]:
default_model = get_default_pydantic_model("medication")

def fix_model_answers(model_answers):
    try:
        return json.dumps(fix_json(model_answers))
    except:
        return str(default_model.model_dump_json())

In [55]:
for i in df_broken.index:
    df.at[i, "model_answers"] = fix_model_answers(df.at[i, "model_answers"])

In [59]:
for idx, (i, j) in enumerate(zip(df["model_answers"], df["text"])):
    try:
        print(json.loads(i))
    except:
        print(f"Failed to load {idx} {j}")

{'medications': [{'name': 'Prednison', 'dose': 100.0, 'dose_unit': 'mg', 'morning': 1.0, 'noon': 0.0, 'evening': 0.0, 'night': 0.0, 'extra': '24.09. - 07.10.2016'}, {'name': 'Prednison', 'dose': 80.0, 'dose_unit': 'mg', 'morning': 1.0, 'noon': 0.0, 'evening': 1.0, 'night': 0.0, 'extra': '08.10. - 14.10.2016'}, {'name': 'Prednison', 'dose': 60.0, 'dose_unit': 'mg', 'morning': 1.0, 'noon': 0.0, 'evening': 1.0, 'night': 0.0, 'extra': '15.10. - 21.10.2016'}, {'name': 'Prednison', 'dose': 40.0, 'dose_unit': 'mg', 'morning': 1.0, 'noon': 0.0, 'evening': 0.0, 'night': 0.0, 'extra': '22.10.2016 bis auf weiteres'}, {'name': 'Pantozol', 'dose': 40.0, 'dose_unit': 'mg', 'morning': 1.0, 'noon': 0.0, 'evening': 1.0, 'night': 0.0, 'extra': 'für die Dauer der Prednison-Behandlung'}]}
{'medications': [{'name': 'Floxal AT', 'dose': 4.0, 'dose_unit': 'tablet', 'morning': 0.0, 'noon': 0.0, 'evening': 0.0, 'night': 1.0, 'extra': '5 Tage'}, {'name': 'Vitamine A AS', 'dose': 1.0, 'dose_unit': 'tablet', 'mor

In [57]:
dfs = []
for idx, (answer, text) in enumerate(zip(df["model_answers"], df["text"])):
    try:
        answer = json.loads(answer)
    except:
        answer = default_model.json()
    medications = answer["medications"]
    for med in medications:
        med["text"] = text
        med["id"] = idx
        dfs.append((med))
res = pd.DataFrame(dfs)

TypeError: string indices must be integers

In [ ]:
results.to_csv(paths.RESULTS_PATH/"medication/labels.csv")

AttributeError: 'dict' object has no attribute 'to_csv'